ACTUALIZA LAS PROPIEDADES QUE TIENEN MIDIFICACIONES

In [18]:
import ee
ee.Authenticate()

True

In [19]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [20]:
ee.Initialize()

In [21]:
ruta_catastro='projects/ee-bismarksr17/assets/LOTES_CANHA_2024'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI-2024-01'

CODIGO_PROPIEDAD='unidad_01'
NOM_PROPIEDAD='unidad_02'
CODIGO_CANHERO='unidad_03'
NOM_CANHERO='unidad_04'

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='area'

In [22]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

## Eliminar propiedades sobra

In [26]:
# props que presentan irregularidades
lista_eliminar = [106, 622, 1080, 1172, 1302, 1413, 1480, 1540, 1626, 1986,
    1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
    1998, 1999, 2001, 2002, 1421, 1962, 1289, 988, 986, 1446,
    1454, 637, 15, 636, 92, 1735, 880, 328, 19, 201, 114, 437,
    216, 1447, 365, 47, 123, 249, 132, 17, 1491, 1015, 85, 325,
    367, 491, 478, 1081, 744, 752, 313, 616, 1265, 1482, 1743]

In [7]:
df_intersects = gpd.read_file('INTERSECT_2.shp')

In [28]:
df_data = pd.read_excel('DATA_EST.xlsx')

### Eliminar props de DATA_EST.xlsx

In [29]:
df_data = df_data[~df_data['cod_prop'].isin(lista_eliminar)]

In [30]:
df_data.to_excel('data_new.xlsx')

### Eliminar props de INTERSECTS.shp

In [14]:
lista_eliminar = [str(i) for i in lista_eliminar]

In [15]:
df_intersects = df_intersects[~df_intersects['unidad_01'].isin(lista_eliminar)]

In [23]:
ruta_fases = 'cods_2024.xlsx'

In [24]:
df = pd.read_excel(ruta_fases)

In [25]:
df.head(5)

,unidad_01
0,1
1,2
2,3
3,4
4,5


In [26]:
lista_props = list(df['unidad_01'])

In [27]:
len(lista_props)

1600

In [28]:
print(lista_props)

[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 53, 54, 55, 57, 61, 62, 63, 66, 67, 68, 71, 72, 74, 76, 78, 79, 80, 83, 84, 85, 89, 92, 93, 94, 96, 98, 100, 103, 104, 105, 106, 109, 110, 111, 112, 113, 114, 116, 117, 118, 119, 122, 123, 125, 126, 127, 128, 129, 132, 133, 135, 136, 137, 139, 140, 141, 142, 145, 146, 147, 149, 150, 155, 156, 159, 161, 162, 164, 168, 171, 172, 173, 174, 176, 178, 179, 182, 183, 184, 185, 186, 187, 188, 189, 195, 197, 198, 201, 205, 206, 209, 211, 212, 213, 216, 217, 218, 219, 220, 221, 222, 225, 226, 227, 228, 229, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 246, 247, 249, 250, 251, 253, 256, 257, 258, 259, 260, 261, 263, 264, 265, 267, 268, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 297, 298, 299, 300, 301, 302, 303, 304, 305, 30

In [61]:

lista_props = [2107]
#lista_props = [2075]

In [63]:
len(lista_props)

1

In [29]:
df_intersects = gpd.read_file('INTERSECT_3.shp')

In [64]:
df_intersects.shape

(940722, 28)

In [65]:
contador = 0
for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_gdf(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    intersect = overlay(lotes_local, vector_local, how="intersection")
    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    
    # Cargar el archivo de Excel existente
    wb = openpyxl.load_workbook('DATA_EST.xlsx')
    # Seleccionar la hoja de trabajo
    ws = wb['data']
    
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    wb.save('DATA_EST.xlsx')
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
    
print('FIN......!')

inicio: 2107


EEException: Image.clip: The geometry for image clipping must not be empty.

In [66]:
len(df_intersects)

940722

In [70]:
df_intersects.to_file("INTERSECT.shp", driver="ESRI Shapefile")

In [69]:
df_intersects['area_2'].sum()

53881.632062450626